In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
xls = pd.ExcelFile('data/ProblemCData.xlsx')
energy_data = pd.read_excel(xls, 'seseds')
variables = pd.read_excel(xls, 'msncodes')

In [ ]:
energy_data.head()

In [ ]:
energy_data['MSN'].nunique()

In [ ]:
variables['MSN'].nunique()

In [ ]:
variables.head()

In [ ]:
energy_data.info()

In [ ]:
variable_columns = energy_data['MSN'].unique()
variable_columns[:5]

In [ ]:
energy_data['StateCode'].unique()

In [ ]:
AZ_data = energy_data[energy_data['StateCode'] == 'AZ']
CA_data = energy_data[energy_data['StateCode'] == 'CA']
NM_data = energy_data[energy_data['StateCode'] == 'NM']
TX_data = energy_data[energy_data['StateCode'] == 'TX']

In [ ]:
data = {}
data['AZ'] = AZ_data.pivot(index='Year', columns='MSN', values='Data')
data['TX'] = TX_data.pivot(index='Year', columns='MSN', values='Data')
data['CA'] = CA_data.pivot(index='Year', columns='MSN', values='Data')
data['NM'] = NM_data.pivot(index='Year', columns='MSN', values='Data')
data['AZ']

In [ ]:
plt.plot(data['AZ']['ABICB'], color='red')
plt.show()

In [ ]:
sns.heatmap(data['AZ'].isnull())
plt.show()

In [ ]:
variables[variables['MSN'] == 'PAACK']

In [ ]:
energy_data[energy_data['MSN'] == 'PAACK']

In [ ]:
xlsx = pd.ExcelFile('data/table5.xlsx')
state_CO2_data = pd.read_excel(xlsx, 'Sheet1')
state_CO2_data.head(30)

In [ ]:
# This code works, but I didn't merge it in time
xlsx = pd.ExcelFile('data/table5.xlsx')
state_CO2_data2 = pd.read_excel(xlsx, 'Sheet1', skiprows=5, skipfooter=2)
state_CO2_data2 = state_CO2_data2.transpose()
state_CO2_data2.columns = state_CO2_data2.loc['State']
state_CO2_data2 = state_CO2_data2.drop('State').drop('Percent').drop('Absolute')
state_CO2_data2

In [ ]:
state_CO2_data.dropna(inplace=True)

In [ ]:
cols = state_CO2_data.columns

In [ ]:
state_CO2_data_trimmed = state_CO2_data[cols[:17]]

In [ ]:
state_CO2_data_trimmed

In [ ]:
TX_CO2_data = pd.DataFrame()

In [ ]:
useful_cols = cols[1:17]

In [ ]:
state_CO2_data_trimmed[state_CO2_data_trimmed['Unnamed: 0'] == 'State'][useful_cols]

In [ ]:
years = state_CO2_data_trimmed[state_CO2_data_trimmed['Unnamed: 0'] == 'State'][useful_cols].values
TX_emissions = state_CO2_data_trimmed[state_CO2_data_trimmed['Unnamed: 0'] == 'Texas'][useful_cols].values

In [ ]:
years

In [ ]:
TX_emissions = TX_emissions[0]

In [ ]:
years = years[0]

In [ ]:
TX_CO2_data['Year'] = years
TX_CO2_data['Emissions per Capita'] = TX_emissions

In [ ]:
TX_CO2_data

In [ ]:
CA_emissions = state_CO2_data_trimmed[state_CO2_data_trimmed['Unnamed: 0'] == 'California'][useful_cols].values
CA_emissions = CA_emissions[0]

NM_emissions = state_CO2_data_trimmed[state_CO2_data_trimmed['Unnamed: 0'] == 'New Mexico'][useful_cols].values
NM_emissions = NM_emissions[0]

AZ_emissions = state_CO2_data_trimmed[state_CO2_data_trimmed['Unnamed: 0'] == 'Arizona'][useful_cols].values
AZ_emissions = AZ_emissions[0]

CA_CO2_data = pd.DataFrame({'Year': years, 'Emissions per Capita': CA_emissions})
NM_CO2_data = pd.DataFrame({'Year': years, 'Emissions per Capita': NM_emissions})
AZ_CO2_data = pd.DataFrame({'Year': years, 'Emissions per Capita': AZ_emissions})

In [ ]:
AZ_CO2_data

In [ ]:
plt.plot(years, CA_CO2_data['Emissions per Capita'], color='blue', label='California')
plt.plot(years, TX_CO2_data['Emissions per Capita'], color='red', label='Texas')
plt.plot(years, AZ_CO2_data['Emissions per Capita'], color='orange', label='Arizona')
plt.plot(years, NM_CO2_data['Emissions per Capita'], color='yellow', label='New Mexico')
plt.xlabel('Year')
plt.ylabel('CO2 emissions per capita')
plt.title('CO2 emissions for e')
plt.legend()
plt.show()

In [ ]:
data['AZ']['ELISB']

In [ ]:
import functools

descriptions = variables['Description'].str.lower().str.replace('.', '').str.strip()
def search_vars(*args):
    masks = [descriptions.str.contains(arg.lower()) for arg in args]
    mask = functools.reduce(lambda x, y: x & y, masks)
    return list(set(descriptions[mask]))
search_vars('waste')

In [ ]:
import re
all_sources = set()
all_sectors = set()
for description in descriptions:
    m = re.match('(.*) consumed by the (.*)', description)
    if m and 'factor' not in description:
        all_sources.add(m.group(1))
        all_sectors.add(m.group(2))

In [ ]:
print(variables['MSN'].str[2:4].unique())
all_sectors = {
    'AC': 'transportation',
    'CC': 'commercial',
    'EI': 'electric',
    'EG': 'electric',
    'IC': 'industrial',
    'RC': 'residential',
    'HC': 'residential and commercial',
}
all_sources_shrt = set(variables['MSN'].str[0:2].unique())

- [Analysis of Texas](https://www.eia.gov/state/analysis.php?sid=TX)

- [CSV meta documentation](https://www.eia.gov/state/seds/sep_use/total/csv/use_csv_doc.pdf)

- [Petroleum documentation](https://www.eia.gov/state/seds/sep_fuel/notes/use_petrol.pdf)

- [Renewable documentation](https://www.eia.gov/state/seds/sep_fuel/notes/use_renew.pdf)

In [ ]:
petroleum_energy = {
    "AB": "aviation gasoline blending components", # only has statistics for sector="transportation", naturally
    "AV": "aviation gasoline",
    "CL": "coal", # note that we are coal consumed at coke plants. That gets counted in source="petroleum coke"
    "DF": "distillate fuel oil", # sector="industrial" includes refinery fuel and non-refinery fuel
    "JF": "jet fuel", # only sector="transportation", naturally. Includes kerosene-type and naptha-type jet fuel
    "KS": "kerosene",
    "LG": "liquified petroleum gas",
    "MB": "motor gasoline blending",
    "MG": "motor gasoline",
    "FS": "petrochemical feedstocks, still gas", # burned as refinery fuel. Only sector="industrial", naturally
    "JN": "naphtha-type jet fuel",
    "JK": "kerosene-type jet fuel",
    "NA": "natural gasoline",
    "NG": "natural gas",
    "PC": "petroleum coke",
    "RF": "residual fuel oil",
    "SG": "still gas", # make sure this does not include FS
}
petroleum_other = {
    "AR": "asphalt and road oil",
    "CC": "coal coke", # only has imports and exports
    "CO": "crude oil", # only consumed by sector="industrial", naturally
    'DK': 'distillate fuel oil and kerosene-type jet fuel',
    'FF': 'fossil fuels', # sum of other things we are measuring
    "LU": "lubricants",
    "FN": "petrochemical feedstocks, naphtha less than 401 degrees f,",
    "FO": "petrochemical feedstocks, other oils equal to or greater than 401 degrees f,",
    'MM': 'motor gasoline total consumption excluding fuel ethanol.',
    "MS": "miscellaneous petroleum products",
    "NN": 'Natural gas (excluding supplemental gaseous fuels)',
    "P1": 'Asphalt and road oil, aviation gasoline, kerosene, lubricants, and "other petroleum products"',
    "P5": 'Other petroleum products (SG and PC consumed as process fuel and AB, MB, PP, and UO consumed as intermediate products).',
    'PA': "all petroleum products",
    'PM': "all petroleum products total consumption excluding fuel ethanol",
    "PO": "other petroleum products",
    "PP": "pentanes plus",
    "SF": "supplemental gaseous fuels",
    "SN": "special napthas",
    'UO': 'unfinished oils',
    'WX': "waxes",
}

renewable_energy = {
    "EM": "fuel ethanol (excluding denaturant)",
    # Fuel ethanol contains a small amount of denaturant, which is added to make thefinished product unsuitable for human consumption. Fuel ethanol denaturantis typically natural gasoline (pentanes plus) or conventional gasoline. Thesevolumes  are  already  accounted  for  under  petroleum.  Therefore,  to  avoiddouble-counting,  and  to  separately  identify  the  renewable  content  of  fuelethanol,  EIA  estimates  the  Btu  content  of  fuel  ethanol  excluding  denaturantconsumed by the United States. 
    "GE": "geothermal energy",
    # note that geothermal variables are 'different'
    # DGECCB	Direct use of geothermal energy and heat pumps in the commercial sector.	Billion Btu
    "NU": "nuclear",
    "PL": "plant condensate", # not in documentation. what is this exactly?
    'HY': "hydroelectricity",
    "SO": "photovoltaic and solar thermal energy",
    # consumption data is reported for residential and commercial sectors together
    'WD': "wood",
    'WS': 'waste', # consider "wood and waste" instead
    'WY': "wind",
}

renewable_other = {
    'EN': 'fuel ethanol (including denaturant)', # see EM
    'RE': 'renewables',
    'WW': 'wood and waste',
    'BM': 'biomass', # sum of other things we are measuring
    'GO': 'weird combination',
    # GOCCB	Geothermal energy and hydroelectricity consumed in the commercial sector.
    # GORCB	Geothermal and solar energy consumed in the residential sector.
    'RO': 'renewable energy production, other than fuel ethanol',
}

electric_energy = {
    'ES': 'electricity'
}

other_other = {
    'PE': 'primary energy',
    'TE': 'total energy',
    'TN': 'primary energy and electricity',
    'US': 'unfractionated stream',
    'GD': 'GDP',
    'EL': 'electricity (im|ex)ports',
    'LO': 'energy losses',
    'TP': 'resident population',
}

In [ ]:
relevant_sources = functools.reduce(lambda x, y: x | y, [
    set(dct.keys())
    for dct in [petroleum_energy, renewable_energy, electric_energy]
])
print(len(relevant_sources), len(relevant_sources) * 5)

In [ ]:
import collections

only_eg = {'AC', 'CC', "EI", "IC", "RC"} # only used for electrical power generation

zeros = collections.defaultdict(set, {
    'EM': {'EI', 'EG', 'RC'}, # not used for electrical power generation nor residential sector
    'DF': {'EI', 'EG'}, # not used for electrical power generation
    'WY': only_eg,
    'NU': only_eg,
    'SO': only_eg,
    'NG': {'EG'},
    'KS': {'AC', 'EI'}
})

relevant_vars = []
for source in relevant_sources:
    for sector in all_sectors:
        this_var = source + sector + 'B'
        these_vars = list(variables[variables['MSN'].str.startswith(this_var)]['MSN'])
        if these_vars:
            relevant_vars.append(this_var)

In [ ]:
relevant_vars

In [ ]:
energy_profile_vars = list(petroleum_energy.keys()) + list(electric_energy.keys()) + list(renewable_energy.keys())

In [ ]:
energy_profile_vars

In [ ]:
def trunc(x):
    return x[:2]

In [ ]:
energy_profile_full_vars = []
for column in data['AZ'].columns:
    if(trunc(column) in energy_profile_vars):
        energy_profile_full_vars.append(column)

In [ ]:
len(energy_profile_full_vars)

In [ ]:
data['AZ'][energy_profile_full_vars]

In [ ]:
rf_training_data_AZ = data['AZ'][(data['AZ'].index <= 2009) & (data['AZ'].index >= 2000)]

In [ ]:
rf_training_data_AZ

In [ ]:
AZ_CO2_data['Emissions per Capita'].loc[:10]

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
random_forest = RandomForestRegressor(n_estimators=1000, n_jobs=4)
random_forest.fit(rf_training_data_AZ, AZ_CO2_data['Emissions per Capita'].iloc[:10])

In [ ]:
random_forest.feature_importances_

In [ ]:
feature_importance_dict = dict(zip(energy_profile_full_vars, random_forest.feature_importances_))

In [ ]:
max(feature_importance_dict, key=feature_importance_dict.get)

In [ ]:
top_50_feats = sorted(feature_importance_dict.items(), key=lambda x: x[1])[-50:]

In [ ]:
dict(top_50_feats)

In [ ]:
plt.figure(figsize=(10,15))
plt.title('Top ten features in random forest model')
plt.bar(range(len(dict(top_50_feats[-10:]))), dict(top_50_feats[-50:]).values(), align='center')
plt.xticks(range(len(dict(top_50_feats[-50:]))), list(dict(top_50_feats[-50:]).keys()))
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.show()